In [ ]:
!pip install dash tweepy vaderSentiment plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 64.5 MB/s eta 0:00:00


In [ ]:
import datetime as dt

import pandas as pd
import tweepy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# ─── Configuration ──────────────────────────────────────────────────────────────
# Replace with your actual Twitter Bearer Token
# You can obtain this from your Twitter Developer account.
BEARER_TOKEN = "yUXi5E4Bzap3OyuJzIbDVMsXp"

# Initialize Twitter client and sentiment analyzer
client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=True)
analyzer = SentimentIntensityAnalyzer()

# ─── Data Fetching & Analysis ───────────────────────────────────────────────────
def fetch_and_analyze(query: str, max_tweets: int = 100):
    """
    Fetch recent tweets matching `query` and compute sentiment scores.
    Returns a DataFrame with columns: created_at, text, sentiment_score.
    """
    tweets = client.search_recent_tweets(
        query=query + " -is:retweet lang:en",
        tweet_fields=["created_at","text"],
        max_results=100
    )

    records = []
    for t in tweets.data or []:
        ts = analyzer.polarity_scores(t.text)["compound"]
        records.append({
            "created_at": t.created_at,
            "text": t.text,
            "sentiment_score": ts
        })

    df = pd.DataFrame(records)
    if df.empty:
        return df

    # Round timestamps to nearest minute for grouping
    df["minute"] = df["created_at"].dt.floor("T")
    return df

# ─── Dash App Layout ────────────────────────────────────────────────────────────
app = dash.Dash(__name__)
server = app.server  # for deployment

app.layout = html.Div([
    html.H1("Twitter Sentiment Dashboard"),
    dcc.Input(
        id="query-input",
        type="text",
        value="openai",
        placeholder="Enter search term",
        style={"width": "300px", "marginRight": "10px"}
    ),
    html.Button("Update", id="update-button"),

    html.Div(id="metrics", style={"display": "flex", "gap": "50px", "marginTop": "20px"}),

    dcc.Graph(id="sentiment-time-series")
], style={"padding": "20px"})


# ─── Callbacks ──────────────────────────────────────────────────────────────────
@app.callback(
    [Output("metrics", "children"),
     Output("sentiment-time-series", "figure")],
    [Input("update-button", "n_clicks")],
    [dash.State("query-input", "value")]
)
def update_dashboard(n_clicks, query):
    if not query:
        return [], px.line()

    df = fetch_and_analyze(query)
    if df.empty:
        return [
            html.Div(["No tweets found"], style={"color": "red"})
        ], px.line()

    # Compute overall metrics
    avg_sent = df["sentiment_score"].mean()
    pos_pct = (df["sentiment_score"] > 0).mean() * 100
    neg_pct = (df["sentiment_score"] < 0).mean() * 100

    metrics = [
        html.Div([
            html.H4("Average Sentiment"),
            html.P(f"{avg_sent:.3f}")
        ]),
        html.Div([
            html.H4("Positive %"),
            html.P(f"{pos_pct:.1f}%")
        ]),
        html.Div([
            html.H4("Negative %"),
            html.P(f"{neg_pct:.1f}%")
        ])
    ]

    # Time-series aggregation
    ts = df.groupby("minute")["sentiment_score"].mean().reset_index()

    fig = px.line(
        ts,
        x="minute",
        y="sentiment_score",
        title=f"Sentiment Over Time for '{query}'",
        labels={"minute": "Time", "sentiment_score": "Avg Sentiment"}
    )

    return metrics, fig


# ─── Main ───────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    app.run(debug=True)

<IPython.core.display.Javascript object>